In [1]:
print("fsdf")

fsdf


In [2]:
import os, csv, random
from datetime import datetime, timedelta

# Ensure file path is valid in current container
fname = "sample_ohlc.csv"

rows = []
now = datetime.utcnow()
start = now - timedelta(hours=100)
price = 64000.0

for i in range(100):
    ts = int((start + timedelta(hours=i)).timestamp() * 1000)  # ms epoch
    open_p = price
    high_p = open_p + random.uniform(50, 300)
    low_p = open_p - random.uniform(50, 300)
    close_p = random.uniform(low_p, high_p)
    vol = random.uniform(10, 100)
    rows.append([ts, open_p, high_p, low_p, close_p, vol, "BTCUSDT"])
    price = close_p

with open(fname, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["timestamp", "open", "high", "low", "close", "volume", "symbol"])
    writer.writerows(rows)

os.path.abspath(fname)


/var/folders/4l/3_f614jn2dl80g90y__djfgr0000gp/T/ipykernel_72132/2360824190.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


'/Users/myadav503/Desktop/1 Projects/2 Personal/FinAI/TJ/sample_ohlc.csv'

In [19]:
import os
import time
import hmac
import hashlib
import json
from urllib.parse import urlencode
import requests

API_KEY    = 'QuNWkG8HsnHELwdcDWZgPWemINWQPJ'
API_SECRET = 'Qbt03P2E07cH85ExjJH0mcXW45Fb49CVaNrRXoA75Bj6sEQnj6foMpUV1Npw'
BASE_URL   = "https://api.india.delta.exchange"

# TODO: Set these values from your own account
# API_KEY = os.getenv("DELTA_API_KEY", "YOUR_API_KEY")
# API_SECRET = os.getenv("DELTA_API_SECRET", "YOUR_API_SECRET")
USER_AGENT = "delta-python-script"

def generate_signature(secret: str, message: str) -> str:
    """Generate HMAC SHA256 signature for the given message."""
    return hmac.new(secret.encode(), message.encode(), hashlib.sha256).hexdigest()

def call_private_api(method: str, path: str, params: dict | None = None,
                     body: dict | None = None) -> dict:
    """
    Send an authenticated request to DeltaExchange.
    - method: HTTP method ('GET', 'POST', etc.)
    - path: endpoint path (e.g. '/v2/fills')
    - params: query parameters (dict)
    - body: JSON serialisable body for non‑GET requests
    """
    if params is None:
        params = {}
    if body is None:
        payload = ""
    else:
        # DeltaExchange expects the body string (no spaces)
        payload = json.dumps(body, separators=(",", ":"))

    # Build the query string
    query_string = f"?{urlencode(params, doseq=True)}" if params else ""

    # Timestamp in seconds (Delta requires this header within ±5 seconds):contentReference[oaicite:4]{index=4}.
    timestamp = str(int(time.time()))

    # Prepare prehash string: method + timestamp + path + query_string + payload
    message = f"{method}{timestamp}{path}{query_string}{payload}"
    signature = generate_signature(API_SECRET, message)

    headers = {
        "api-key": API_KEY,
        "timestamp": timestamp,
        "signature": signature,
        "User-Agent": USER_AGENT,
        "Accept": "application/json",
        "Content-Type": "application/json",
    }

    url = BASE_URL + path
    # For GET requests, pass payload via params; otherwise via data
    response = requests.request(
        method=method,
        url=url,
        params=params,
        data=payload if method != "GET" else None,
        headers=headers,
        timeout=(5, 20),
    )
    # Raise if non‑success status (e.g. 401, 429)
    response.raise_for_status()
    return response.json()

def fetch_all_pages(path: str, extra_params: dict | None = None) -> list:
    """
    Iterate through all pages of a paginated DeltaExchange endpoint.
    Returns a list of all records.
    """
    if extra_params is None:
        extra_params = {}

    page_size = extra_params.get("page_size", 100)
    cursor = None
    all_records: list = []

    while True:
        params = extra_params.copy()
        params["page_size"] = page_size
        if cursor:
            params["after"] = cursor  # use after cursor for next page:contentReference[oaicite:5]{index=5}.
        response = call_private_api("GET", path, params=params)
        # On success, response dict contains 'result' and 'meta'
        records = response.get("result", [])
        all_records.extend(records)

        # Stop if no pagination metadata
        meta = response.get("meta", {})
        cursor = meta.get("after")
        if not cursor:
            break
    return all_records

def get_all_fills(start_time: int | None = None, end_time: int | None = None,
                  product_ids: str | None = None, contract_types: str | None = None,
                  page_size: int = 100) -> list:
    """
    Fetch all fills (executions/trades) for your account.
    - start_time/end_time: microsecond epoch timestamps to filter by time:contentReference[oaicite:6]{index=6}.
    - product_ids: comma‑separated product IDs, optional:contentReference[oaicite:7]{index=7}.
    - contract_types: comma‑separated contract types, optional:contentReference[oaicite:8]{index=8}.
    """
    params: dict = {"page_size": page_size}
    if product_ids:
        params["product_ids"] = product_ids
    if contract_types:
        params["contract_types"] = contract_types
    if start_time:
        params["start_time"] = start_time
    if end_time:
        params["end_time"] = end_time
    return fetch_all_pages("/v2/fills", params)

def get_all_order_history(start_time: int | None = None, end_time: int | None = None,
                          product_ids: str | None = None, contract_types: str | None = None,
                          order_types: str | None = None, page_size: int = 100) -> list:
    """
    Fetch all closed/cancelled orders from your account:contentReference[oaicite:9]{index=9}.
    - order_types: comma‑separated order types (e.g. 'market,limit,stop_limit'):contentReference[oaicite:10]{index=10}.
    """
    params: dict = {"page_size": page_size}
    if product_ids:
        params["product_ids"] = product_ids
    if contract_types:
        params["contract_types"] = contract_types
    if order_types:
        params["order_types"] = order_types
    if start_time:
        params["start_time"] = start_time
    if end_time:
        params["end_time"] = end_time
    return fetch_all_pages("/v2/orders/history", params)

if __name__ == "__main__":
    # Example: download all fills (trades) for your account
    try:
        fills = get_all_fills(page_size=100)
        print(f"Fetched {len(fills)} fills.")
        # Inspect the first fill
        if fills:
            print("First fill:", fills[0])
    except requests.HTTPError as err:
        print("HTTP error:", err)
    except Exception as e:
        print("Error:", e)

    # Example: download all order history
    try:
        orders = get_all_order_history(page_size=100)
        print(f"Fetched {len(orders)} closed/cancelled orders.")
    except Exception as e:
        print("Error fetching orders:", e)


Fetched 309 fills.
First fill: {'commission': '0', 'created_at': '2025-08-25T20:32:41.897796Z', 'fill_type': 'normal', 'id': 'e8457e97ef4e4e649c4fe8c7101bcc66', 'meta_data': {'commission_deto': '0', 'commission_deto_in_settling_asset': '0', 'effective_commission_rate': '0.0005', 'ip': None, 'is_commission_on_notional': True, 'liquidation_fee_deto': '0', 'liquidation_fee_deto_in_settling_asset': '0', 'liquidation_fee_in_settling_asset': '0', 'mark': '188.30345198', 'new_position': {'bankruptcy_price': None, 'commission': '0.00000000000000000000', 'liquidation_price': None, 'margin': '0.00000000', 'meta_data': {'open_fills': []}, 'realized_pnl': '-3.33450000', 'size': 0}, 'order_price': '169.745', 'order_size': 5, 'order_type': 'market_order', 'order_unfilled_size': '0', 'rack_commision': '0.55544075', 'source': 'web', 'spot': '188.35666667', 'tfc_used_for_commission': '0.00', 'tfc_used_for_liquidation_fee': '-0', 'total_commission_in_settling_asset': '0', 'total_liquidation_fee_in_settl

In [22]:
orders[1]

{'side': 'sell',
 'created_at': '2025-08-25T20:21:32.842227Z',
 'average_fill_price': '188.285',
 'trail_amount': None,
 'client_order_id': None,
 'state': 'closed',
 'time_in_force': 'gtc',
 'product': {'contract_type': 'perpetual_futures',
  'contract_unit_currency': 'SOL',
  'contract_value': '1',
  'id': 14823,
  'notional_type': 'vanilla',
  'quoting_asset': {'minimum_precision': 2, 'precision': 8, 'symbol': 'USD'},
  'settling_asset': {'minimum_precision': 2, 'precision': 8, 'symbol': 'USD'},
  'spot_index': {'symbol': '.DESOLUSD'},
  'symbol': 'SOLUSD',
  'tick_size': '0.0001',
  'underlying_asset': {'minimum_precision': 4,
   'precision': 8,
   'symbol': 'SOL'}},
 'meta_data': {'avg_exit_price': '188.285',
  'cashflow': '-3.3345',
  'edit_timestamp': '2025-08-25 20:21:47.987021Z',
  'entry_price': '188.95190000',
  'exit_price_with_size': [{'price': '188.28500000', 'size': '5'}],
  'ip': None,
  'otc': False,
  'pnl': '-3.3345',
  'roe': '-28.23575735412028140495014869',
  'sou

In [1]:
import time
import hmac
import hashlib
import json
from urllib.parse import urlencode
import requests

# Use the correct environment:
# BASE_URL = "https://api.india.delta.exchange"

API_KEY    = "VJOGAFCUKX7S2XIX"
API_SECRET = "Rz1miB9WrT4AOxYDAaxbAaJe2eCZAj"

API_KEY    = 'QuNWkG8HsnHELwdcDWZgPWemINWQPJ'
API_SECRET = 'Qbt03P2E07cH85ExjJH0mcXW45Fb49CVaNrRXoA75Bj6sEQnj6foMpUV1Npw'

BASE_URL   = "https://api.india.delta.exchange"
USER_AGENT = "python-3.10"  # Delta suggests using your language version:contentReference[oaicite:6]{index=6}

def generate_signature(secret, message):
    return hmac.new(secret.encode(), message.encode(), hashlib.sha256).hexdigest()

def delta_request(method, path, params=None, body=None):
    if params is None:
        params = {}
    timestamp = str(int(time.time()))
    body_str = json.dumps(body, separators=(",", ":")) if body else ""
    query = "?" + urlencode(params, doseq=True) if params else ""
    message = method + timestamp + path + query + body_str
    signature = generate_signature(API_SECRET, message)

    headers = {
        "api-key": API_KEY,
        "timestamp": timestamp,
        "signature": signature,
        "User-Agent": USER_AGENT,
        "Accept": "application/json",
        "Content-Type": "application/json",
    }

    url = BASE_URL + path
    response = requests.request(method, url, params=params, data=body_str if method != "GET" else None, headers=headers)
    response.raise_for_status()  # will show 401 details if auth fails
    return response.json()

# Example: Fetch all fills (executed trades) with pagination
def get_all_fills(page_size=100):
    fills = []
    cursor = None
    while True:
        params = {"page_size": page_size}
        if cursor:
            params["after"] = cursor
        result = delta_request("GET", "/v2/fills", params=params)
        fills.extend(result.get("result", []))
        cursor = result.get("meta", {}).get("after")
        if not cursor:
            break
    return fills

try:
    all_fills = get_all_fills()
    print(f"Total fills fetched: {len(all_fills)}")
except requests.HTTPError as err:
    print("HTTP error:", err.response.text)


Total fills fetched: 511


In [18]:
all_fills[2]

{'commission': '0.04459265',
 'created_at': '2025-08-25T20:21:32.681147Z',
 'fill_type': 'normal',
 'id': 'cde29e7eb62b48d6aa5d33eec099d675',
 'meta_data': {'commission_deto': '0',
  'commission_deto_in_settling_asset': '0',
  'effective_commission_rate': '0.0002',
  'ip': '103.111.180.131',
  'is_commission_on_notional': True,
  'liquidation_fee_deto': '0',
  'liquidation_fee_deto_in_settling_asset': '0',
  'liquidation_fee_in_settling_asset': '0',
  'mark': '189.05251347',
  'new_position': {'auto_topup': True,
   'bankruptcy_price': '186.59000125',
   'commission': '0.1115409829473000000000000000',
   'entry_price': '188.95190000',
   'liquidation_price': '187.53476075',
   'margin': '2.36189875',
   'meta_data': {'open_fills': [['buy',
      '1',
      '2025-08-25T20:21:32.693036Z',
      '0.04459265',
      '188.9519000000000000000000000']]},
   'size': 1,
   'total_commission_paid': '0.04459265'},
  'order_price': '188.9519',
  'order_size': '5',
  'order_type': 'limit_order',
  

In [23]:
import os, time, hmac, hashlib, json, requests
from urllib.parse import urlencode

BASE_URL   = "https://api.india.delta.exchange"   # ← India prod, not global/testnet!

def _sign(method: str, path: str, params: dict | None = None, body: dict | None = None):
    """
    Signature = HMAC_SHA256(secret, method + timestamp + path + query_string + body_json)
    - query_string must include the leading '?', and be URL-encoded
    - body_json must be the exact JSON sent on the wire (no spaces)
    """
    ts = str(int(time.time()))
    q  = f"?{urlencode(params, doseq=True)}" if params else ""
    b  = json.dumps(body, separators=(",", ":")) if body else ""
    prehash = f"{method.upper()}{ts}{path}{q}{b}"
    sig = hmac.new(API_SECRET.encode(), prehash.encode(), hashlib.sha256).hexdigest()
    headers = {
        "api-key": API_KEY,
        "signature": sig,
        "timestamp": ts,
        "User-Agent": "python-3.12",
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    return headers, b

def _request(method: str, path: str, params=None, body=None):
    headers, body_json = _sign(method, path, params, body)
    url = f"{BASE_URL}{path}"
    if method.upper() == "GET":
        r = requests.get(url, params=params, headers=headers, timeout=30)
    elif method.upper() == "POST":
        r = requests.post(url, params=params, data=body_json, headers=headers, timeout=30)
    elif method.upper() == "DELETE":
        r = requests.delete(url, params=params, data=body_json, headers=headers, timeout=30)
    else:
        raise ValueError("Unsupported method")
    r.raise_for_status()
    return r.json()

def get_all_orders(states: list[str] | None = None, page_size: int = 200):
    """
    Pulls ALL orders (open/pending/cancelled) from /v2/orders using cursor pagination,
    and ALL historical (closed/filled, etc.) from /v2/orders/history.
    """
    def _page_through(path):
        out = []
        after = None
        while True:
            params = {"page_size": page_size}
            if after: params["after"] = after
            if states: params["states"] = ",".join(states)
            data = _request("GET", path, params=params)
            out.extend(data.get("result", []))
            meta = data.get("meta") or {}
            after = meta.get("after")
            if not after:
                break
        return out

    current = _page_through("/v2/orders")          # open/pending/cancelled
    history = _page_through("/v2/orders/history")  # closed/filled (historical)
    return current + history

# Example usage:
# orders = get_all_orders(states=None)                   # everything
# open_orders = get_all_orders(states=["open","pending"])
# print(len(orders), orders[:2])


In [36]:
# delta_india_client.py
# Authenticated REST client for Delta Exchange India (orders + fills)
# Works with: https://api.india.delta.exchange

import os, time, hmac, hashlib, json, requests
from urllib.parse import urlencode
from typing import Optional, Iterable, Dict, Any, List, Tuple

# API_KEY    = os.getenv("DELTA_API_KEY")    # set in your env
# API_SECRET = os.getenv("DELTA_API_SECRET") # set in your env
BASE_URL   = "https://api.india.delta.exchange"
USER_AGENT = "delta-india-python/1.0"




from datetime import datetime, timezone
from typing import Union, Optional

TimeLike = Union[int, float, str, datetime]

def _to_unix_seconds(t: Optional[TimeLike]) -> Optional[int]:
    """
    Convert various time inputs to UNIX seconds (UTC).
    Accepts:
      - int/float (assumed seconds)
      - datetime (aware = used as-is; naive = assumed UTC)
      - ISO-8601 string (e.g., '2025-08-01', '2025-08-01T10:00:00Z')
    Returns None if t is None.
    """
    if t is None:
        return None
    if isinstance(t, (int, float)):
        return int(t)
    if isinstance(t, datetime):
        if t.tzinfo is None:
            t = t.replace(tzinfo=timezone.utc)
        return int(t.timestamp())
    if isinstance(t, str):
        # allow date-only (interpreted as 00:00:00Z) and full ISO
        s = t.strip()
        if len(s) == 10 and s[4] == "-" and s[7] == "-":  # 'YYYY-MM-DD'
            dt = datetime.fromisoformat(s).replace(tzinfo=timezone.utc)
            return int(dt.timestamp())
        # general ISO; tolerate trailing 'Z'
        dt = datetime.fromisoformat(s.replace("Z", "+00:00"))
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return int(dt.timestamp())
    raise TypeError(f"Unsupported time type: {type(t)}")
def filter_orders_by_time(orders: list[dict], start: Optional[TimeLike] = None, end: Optional[TimeLike] = None) -> list[dict]:
    """
    Return only orders whose created_at is within [start, end] (inclusive).
    - start/end can be UNIX seconds, datetime, or ISO-8601 strings.
    - If only start is provided: keep orders with created_at >= start.
    - If only end is provided: keep orders with created_at <= end.
    """
    start_s = _to_unix_seconds(start)
    end_s   = _to_unix_seconds(end)

    def _created_ts(o: dict) -> Optional[int]:
        s = o.get("created_at")
        if not s:
            return None
        try:
            dt = datetime.fromisoformat(s.replace("Z", "+00:00"))
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=timezone.utc)
            return int(dt.timestamp())
        except Exception:
            return None

    out = []
    for o in orders:
        ts = _created_ts(o)
        if ts is None:
            continue
        if start_s is not None and ts < start_s:
            continue
        if end_s is not None and ts > end_s:
            continue
        out.append(o)
    return out

class DeltaIndiaError(Exception):
    pass

def _assert_keys():
    if not API_KEY or not API_SECRET:
        raise DeltaIndiaError("Missing DELTA_API_KEY / DELTA_API_SECRET environment variables.")

def _now_ts() -> str:
    # Delta expects seconds (int) as string
    return str(int(time.time()))

def _build_query(params: Optional[dict]) -> str:
    if not params:
        return ""
    # NOTE: urlencode must match the exact query you send
    return "?" + urlencode(params, doseq=True)

def _sign(method: str, path: str, params: Optional[dict] = None, body: Optional[dict] = None) -> Tuple[dict, str]:
    """
    Signature = HMAC_SHA256(secret, method + timestamp + path + query + body_json)
    - method: GET/POST/DELETE (upper)
    - timestamp: seconds since epoch (string)
    - query: include leading '?' if present
    - body_json: exact JSON string sent on the wire (minified, no spaces)
    """
    _assert_keys()
    ts = _now_ts()
    q  = _build_query(params)
    b  = json.dumps(body, separators=(",", ":")) if body else ""
    prehash = f"{method.upper()}{ts}{path}{q}{b}"
    sig = hmac.new(API_SECRET.encode(), prehash.encode(), hashlib.sha256).hexdigest()
    headers = {
        "api-key": API_KEY,
        "signature": sig,
        "timestamp": ts,
        "User-Agent": USER_AGENT,
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    return headers, b

def _request(method: str, path: str, params: Optional[dict] = None, body: Optional[dict] = None) -> dict:
    headers, body_json = _sign(method, path, params, body)
    url = f"{BASE_URL}{path}"

    try:
        if method.upper() == "GET":
            r = requests.get(url, params=params, headers=headers, timeout=30)
        elif method.upper() == "POST":
            r = requests.post(url, params=params, data=body_json, headers=headers, timeout=30)
        elif method.upper() == "DELETE":
            r = requests.delete(url, params=params, data=body_json, headers=headers, timeout=30)
        else:
            raise ValueError(f"Unsupported method: {method}")
        # Raise for HTTP errors (401, 403, 429, 5xx)
        r.raise_for_status()
        data = r.json()
        # Delta private endpoints typically return {"success": bool, "result": [...], "meta": {...}} or similar
        if isinstance(data, dict) and data.get("success") is False:
            raise DeltaIndiaError(f"Delta API error: {data}")
        return data
    except requests.HTTPError as e:
        # Bubble up with more context
        msg = e.response.text if e.response is not None else str(e)
        raise DeltaIndiaError(f"HTTPError {e.response.status_code if e.response else ''}: {msg}") from e
    except requests.RequestException as e:
        raise DeltaIndiaError(f"Network error: {e}") from e

# ---------- High-level helpers ----------

def _page_through(path: str, params: Optional[dict] = None, page_size: int = 200, max_pages: Optional[int] = None) -> List[Dict[str, Any]]:
    """
    Generic cursor paginator for Delta India private list endpoints.
    Loops on meta.after until exhausted (or max_pages reached).
    """
    params = dict(params or {})
    out: List[Dict[str, Any]] = []
    after = None
    pages = 0

    while True:
        req_params = dict(params)
        req_params["page_size"] = page_size
        if after:
            req_params["after"] = after

        data = _request("GET", path, params=req_params)
        result = data.get("result", [])
        out.extend(result)

        meta = data.get("meta") or {}
        after = meta.get("after")
        pages += 1

        if not after:
            break
        if max_pages is not None and pages >= max_pages:
            break

    return out

# ---------- Orders ----------

def get_all_orders(
    states: Optional[Iterable[str]] = None,
    product_ids: Optional[Iterable[int]] = None,
    page_size: int = 200,
    max_pages: Optional[int] = None,
) -> List[Dict[str, Any]]:
    """
    Fetch ALL orders = open/pending/cancelled + historical closed orders.
    - states: e.g., ["open","pending","cancelled","closed"] or None for all
    - product_ids: optional filter (list -> comma-separated)
    """
    base_params = {}
    if states:
        base_params["states"] = ",".join(states)
    if product_ids:
        base_params["product_ids"] = ",".join(str(p) for p in product_ids)

    current = _page_through("/v2/orders", base_params, page_size, max_pages)
    history = _page_through("/v2/orders/history", base_params, page_size, max_pages)
    return current + history

# ---------- Fills ----------
def get_all_orders_in_range(
        start: Optional[TimeLike] = None,
        end: Optional[TimeLike] = None,
        **kwargs
    ) -> list[dict]:
        """
        Fetch all orders (open + history) and filter by created_at in [start, end].
        kwargs are passed to get_all_orders (e.g., product_ids, states, page_size).
        """
        all_orders = get_all_orders(**kwargs)
        return filter_orders_by_time(all_orders, start=start, end=end)
def get_all_fills(
    product_ids: Optional[Iterable[int]] = None,
    page_size: int = 200,
    max_pages: Optional[int] = None,
) -> List[Dict[str, Any]]:
    """
    Fetch ALL fills (executions) for your account (AUTH REQUIRED).
    - product_ids: optional filter
    """
    params = {}
    if product_ids:
        params["product_ids"] = ",".join(str(p) for p in product_ids)
    return _page_through("/v2/fills", params, page_size, max_pages)

def fills_by_order_id(fills: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """
    Group fills by order_id: { order_id: [fill, ...] }
    """
    grouped: Dict[str, List[Dict[str, Any]]] = {}
    for f in fills:
        oid = str(f.get("order_id"))
        grouped.setdefault(oid, []).append(f)
    # Sort each list by created_at if present
    for oid in grouped:
        grouped[oid].sort(key=lambda x: x.get("created_at", ""))
    return grouped

# ---------- Convenience: reconcile orders ⇄ fills ----------

def join_orders_fills(orders: List[Dict[str, Any]], fills: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Returns a list of per-order summaries including weighted avg fill price, total filled size, total fees.
    """
    by_order = fills_by_order_id(fills)
    out: List[Dict[str, Any]] = []

    for o in orders:
        oid = str(o.get("id"))
        fs = by_order.get(oid, [])
        total_qty = 0.0
        notional = 0.0
        fee_total = 0.0

        for f in fs:
            # prices/sizes may be strings; coerce safely
            price = float(f.get("price") or 0)
            size  = float(f.get("size") or 0)
            fee   = float(f.get("fee") or 0)
            notional += price * size
            total_qty += size
            fee_total += fee

        avg_price = (notional / total_qty) if total_qty else None

        out.append({
            "order_id": oid,
            "state": o.get("state"),
            "product_symbol": o.get("product_symbol") or (o.get("product", {}) or {}).get("symbol"),
            "side": o.get("side"),
            "submitted_size": o.get("size"),
            "filled_size": total_qty,
            "avg_fill_price": avg_price,
            "fee_total": fee_total,
            "created_at": o.get("created_at"),
            "updated_at": o.get("updated_at"),
            "order_type": o.get("order_type"),
            "reduce_only": o.get("reduce_only"),
            "time_in_force": o.get("time_in_force"),
        })

    return out

# ---------- Example run ----------

if __name__ == "__main__":
    # Set env first:
    #   export DELTA_API_KEY=...
    #   export DELTA_API_SECRET=...
    # Optionally: filter specific products (e.g., SOLUSD = 14823)
    try:
        orders = get_all_orders(states=None, product_ids=None)
        fills  = get_all_fills(product_ids=None)

        print(f"Orders: {len(orders)} | Fills: {len(fills)}")

        joined = join_orders_fills(orders, fills)
        # Print first 3 summaries nicely
        for row in joined[:3]:
            print(row)

    except DeltaIndiaError as e:
        print("Error:", e)


Orders: 828 | Fills: 309
{'order_id': '878398215', 'state': 'cancelled', 'product_symbol': 'SOLUSD', 'side': 'buy', 'submitted_size': 5, 'filled_size': 0.0, 'avg_fill_price': None, 'fee_total': 0.0, 'created_at': '2025-08-26T08:40:13.592281Z', 'updated_at': '2025-08-26T08:40:22.017796Z', 'order_type': 'limit_order', 'reduce_only': False, 'time_in_force': 'gtc'}
{'order_id': '875971359', 'state': 'closed', 'product_symbol': 'SOLUSD', 'side': 'sell', 'submitted_size': 5, 'filled_size': 5.0, 'avg_fill_price': 188.285, 'fee_total': 0.0, 'created_at': '2025-08-25T20:21:32.842227Z', 'updated_at': '2025-08-25T20:32:41.954658Z', 'order_type': 'market_order', 'reduce_only': True, 'time_in_force': 'gtc'}
{'order_id': '875931602', 'state': 'closed', 'product_symbol': 'SOLUSD', 'side': 'buy', 'submitted_size': 5, 'filled_size': 5.0, 'avg_fill_price': 188.9519, 'fee_total': 0.0, 'created_at': '2025-08-25T20:12:58.624013Z', 'updated_at': '2025-08-25T20:21:33.058646Z', 'order_type': 'limit_order', 'r

In [37]:
start_ist = "2025-08-20T00:00:00+05:30"
end_ist   = "2025-08-25T23:59:59+05:30"
orders_ist_window = get_all_orders_in_range(start=start_ist, end=end_ist)


In [39]:
len(orders_ist_window)

325

In [25]:
orders = get_all_orders(states=None)                   # everything
open_orders = get_all_orders(states=["open","pending"])
print(len(orders), orders[:2])

828 [{'bracket_trail_amount': None, 'stop_order_type': None, 'meta_data': {'bracket_stop_trigger_method': 'mark_price', 'ip': '103.111.180.174', 'otc': False, 'source': 'desktop'}, 'bracket_take_profit_limit_price': None, 'order_type': 'limit_order', 'product_id': 14823, 'product': {'contract_type': 'perpetual_futures', 'contract_unit_currency': 'SOL', 'contract_value': '1', 'id': 14823, 'notional_type': 'vanilla', 'quoting_asset': {'minimum_precision': 2, 'precision': 8, 'symbol': 'USD'}, 'settling_asset': {'minimum_precision': 2, 'precision': 8, 'symbol': 'USD'}, 'spot_index': {'symbol': '.DESOLUSD'}, 'symbol': 'SOLUSD', 'tick_size': '0.0001', 'underlying_asset': {'minimum_precision': 4, 'precision': 8, 'symbol': 'SOL'}}, 'time_in_force': 'gtc', 'state': 'cancelled', 'trail_amount': None, 'average_fill_price': None, 'id': 878398215, 'close_on_trigger': 'false', 'stop_price': None, 'paid_commission': '0', 'side': 'buy', 'user_id': 93020796, 'limit_price': '187.5414', 'client_order_id'

In [31]:

all_fills = get_all_fills()                          # every fill
sol_fills = get_all_fills(product_ids=[14823])       # SOLUSD only
by_order = fills_by_order_id(sol_fills)
print(len(all_fills), list(by_order)[:3])

HTTPError: 401 Client Error: Unauthorized for url: https://api.india.delta.exchange/v2/fills?page_size=200

In [33]:
"""
delta_trades.py

Delta Exchange India client + trade assembler.

What you get:
- Authenticated REST client (orders + fills) with HMAC signing.
- Dataclasses for Order, Fill, and Trade.
- A robust assembler that collapses multiple exit legs (TP1/TP2/SL/manual)
  into a single Trade (flat-to-flat), with FIFO PnL and useful summaries.

Requirements:
- Python 3.10+
- requests

Environment:
    export DELTA_API_KEY=...
    export DELTA_API_SECRET=...

Notes:
- Base URL is India: https://api.india.delta.exchange
- Pagination uses cursor meta.after
- We infer "TP" vs "SL" legs:
    * If order.stop_order_type is set → 'sl'
    * Else reduce_only True and price favorable vs. avg entry → 'tp'
    * Else → 'manual'
- Position cycles are detected per symbol by walking orders in time:
    We treat non-reduce-only orders as entries/scale-ins, and reduce-only
    orders as exits/scale-outs. A Trade is complete when position returns to 0.
"""

from __future__ import annotations
import os, time, hmac, hashlib, json, math, requests
from dataclasses import dataclass, field
from typing import Optional, Iterable, Dict, Any, List, Tuple
from urllib.parse import urlencode
from datetime import datetime


# =========================
# Low-level HTTP Client
# =========================

class DeltaIndiaError(Exception):
    """Raised for transport or API errors from Delta Exchange India."""


class DeltaIndiaClient:
    """
    Minimal authenticated client for Delta Exchange India private REST APIs.

    Methods:
        get_orders(...)      -> List[dict]
        get_orders_history(...) -> List[dict]
        get_fills(...)       -> List[dict]
        get_all_orders(...)  -> List[dict]
        get_all_fills(...)   -> List[dict]

    Usage:
        client = DeltaIndiaClient()
        orders = client.get_all_orders()
        fills  = client.get_all_fills()
    """

    BASE_URL = "https://api.india.delta.exchange"
    USER_AGENT = "delta-india-python/1.0"

    def __init__(self, api_key: Optional[str] = None, api_secret: Optional[str] = None):
        self.api_key = api_key or os.getenv("DELTA_API_KEY")
        self.api_secret = api_secret or os.getenv("DELTA_API_SECRET")
        if not self.api_key or not self.api_secret:
            raise DeltaIndiaError("Missing DELTA_API_KEY / DELTA_API_SECRET environment variables or constructor args.")

    # ---------- signing / request ----------

    @staticmethod
    def _now_ts() -> str:
        # Delta expects seconds (int) as string
        return str(int(time.time()))

    @staticmethod
    def _build_query(params: Optional[dict]) -> str:
        if not params:
            return ""
        return "?" + urlencode(params, doseq=True)

    def _sign(self, method: str, path: str, params: Optional[dict] = None, body: Optional[dict] = None) -> Tuple[dict, str]:
        """
        HMAC_SHA256(secret, method + timestamp + path + query + body_json)
        - query: MUST include leading '?' if present
        - body_json: minified JSON string (no spaces)
        """
        ts = self._now_ts()
        q = self._build_query(params)
        b = json.dumps(body, separators=(",", ":")) if body else ""
        prehash = f"{method.upper()}{ts}{path}{q}{b}"
        sig = hmac.new(self.api_secret.encode(), prehash.encode(), hashlib.sha256).hexdigest()
        headers = {
            "api-key": self.api_key,
            "signature": sig,
            "timestamp": ts,
            "User-Agent": self.USER_AGENT,
            "Accept": "application/json",
            "Content-Type": "application/json",
        }
        return headers, b

    def _request(self, method: str, path: str, params: Optional[dict] = None, body: Optional[dict] = None) -> dict:
        headers, body_json = self._sign(method, path, params, body)
        url = f"{self.BASE_URL}{path}"
        try:
            if method.upper() == "GET":
                r = requests.get(url, params=params, headers=headers, timeout=30)
            elif method.upper() == "POST":
                r = requests.post(url, params=params, data=body_json, headers=headers, timeout=30)
            elif method.upper() == "DELETE":
                r = requests.delete(url, params=params, data=body_json, headers=headers, timeout=30)
            else:
                raise ValueError(f"Unsupported method: {method}")
            r.raise_for_status()
            data = r.json()
            if isinstance(data, dict) and data.get("success") is False:
                raise DeltaIndiaError(f"Delta API error: {data}")
            return data
        except requests.HTTPError as e:
            msg = e.response.text if e.response is not None else str(e)
            code = e.response.status_code if e.response is not None else "?"
            raise DeltaIndiaError(f"HTTPError {code}: {msg}") from e
        except requests.RequestException as e:
            raise DeltaIndiaError(f"Network error: {e}") from e

    # ---------- pagination helper ----------

    def _page_through(self, path: str, params: Optional[dict] = None, page_size: int = 200, max_pages: Optional[int] = None) -> List[dict]:
        params = dict(params or {})
        out: List[dict] = []
        after = None
        pages = 0
        while True:
            req_params = dict(params)
            req_params["page_size"] = page_size
            if after:
                req_params["after"] = after
            data = self._request("GET", path, req_params)
            result = data.get("result", [])
            out.extend(result)
            meta = data.get("meta") or {}
            after = meta.get("after")
            pages += 1
            if not after:
                break
            if max_pages is not None and pages >= max_pages:
                break
        return out

    # ---------- public methods ----------

    def get_orders(self, *, states: Optional[Iterable[str]] = None, product_ids: Optional[Iterable[int]] = None,
                   page_size: int = 200, max_pages: Optional[int] = None) -> List[dict]:
        params = {}
        if states:
            params["states"] = ",".join(states)
        if product_ids:
            params["product_ids"] = ",".join(str(p) for p in product_ids)
        return self._page_through("/v2/orders", params, page_size, max_pages)

    def get_orders_history(self, *, states: Optional[Iterable[str]] = None, product_ids: Optional[Iterable[int]] = None,
                           page_size: int = 200, max_pages: Optional[int] = None) -> List[dict]:
        params = {}
        if states:
            params["states"] = ",".join(states)
        if product_ids:
            params["product_ids"] = ",".join(str(p) for p in product_ids)
        return self._page_through("/v2/orders/history", params, page_size, max_pages)

    def get_fills(self, *, product_ids: Optional[Iterable[int]] = None, page_size: int = 200,
                  max_pages: Optional[int] = None) -> List[dict]:
        params = {}
        if product_ids:
            params["product_ids"] = ",".join(str(p) for p in product_ids)
        return self._page_through("/v2/fills", params, page_size, max_pages)

    # convenience
    def get_all_orders(self, **kwargs) -> List[dict]:
        return self.get_orders(**kwargs) + self.get_orders_history(**kwargs)

    def get_all_fills(self, **kwargs) -> List[dict]:
        return self.get_fills(**kwargs)


# =========================
# Domain Records
# =========================

def _to_float(x: Any) -> float:
    try:
        return float(x)
    except Exception:
        return 0.0

def _ts(s: Optional[str]) -> float:
    """Parse ISO8601 to timestamp (seconds)."""
    if not s:
        return 0.0
    try:
        return datetime.fromisoformat(s.replace("Z", "+00:00")).timestamp()
    except Exception:
        return 0.0

@dataclass
class Fill:
    order_id: str
    symbol: str
    side: str              # 'buy' or 'sell'
    price: float
    size: float
    fee: float
    created_at: str

    @staticmethod
    def from_api(d: dict) -> "Fill":
        return Fill(
            order_id=str(d.get("order_id")),
            symbol=d.get("product_symbol") or (d.get("product", {}) or {}).get("symbol") or "",
            side=(d.get("side") or "").lower(),
            price=_to_float(d.get("price")),
            size=_to_float(d.get("size")),
            fee=_to_float(d.get("fee")),
            created_at=d.get("created_at") or "",
        )

@dataclass
class Order:
    id: str
    symbol: str
    side: str              # 'buy' / 'sell'
    size: float
    state: str
    reduce_only: bool
    order_type: str
    created_at: str
    updated_at: str
    stop_order_type: Optional[str] = None
    client_order_id: Optional[str] = None
    bracket_order: bool = False
    meta_data: Dict[str, Any] = field(default_factory=dict)

    @staticmethod
    def from_api(d: dict) -> "Order":
        return Order(
            id=str(d.get("id")),
            symbol=d.get("product_symbol") or (d.get("product", {}) or {}).get("symbol") or "",
            side=(d.get("side") or "").lower(),
            size=_to_float(d.get("size")),
            state=(d.get("state") or "").lower(),
            reduce_only=bool(d.get("reduce_only")),
            order_type=d.get("order_type") or "",
            created_at=d.get("created_at") or "",
            updated_at=d.get("updated_at") or "",
            stop_order_type=d.get("stop_order_type"),
            client_order_id=d.get("client_order_id"),
            bracket_order=bool(d.get("bracket_order")),
            meta_data=d.get("meta_data") or {},
        )

@dataclass
class TradeLeg:
    """One exit leg in a trade (TP/SL/manual) with its fills aggregated."""
    label: str             # 'tp1', 'tp2', ..., or 'sl', or 'manual'
    order: Order
    fills: List[Fill]
    filled_size: float
    avg_price: float
    fee: float
    realized_pnl: float    # pnl contributed by this leg (USD)

@dataclass
class Trade:
    """
    A flat-to-flat cycle per symbol (multi-entry + multi-exit supported).
    We track FIFO across fills to compute realized PnL.

    Attributes:
        symbol: product symbol (e.g., 'SOLUSD')
        direction: 'long' or 'short'
        entry_orders: list of non-reduce-only orders that opened/added
        entry_fills: all entry fills (buy for long, sell for short)
        exit_legs: list of TradeLegs in time sequence
        size: maximum absolute position size during the cycle
        avg_entry: weighted average entry across entry_fills
        avg_exit: weighted average of all exit fills (None if still open)
        total_fee: sum of fees across all fills in the trade
        realized_pnl: realized PnL for the whole cycle (USD)
        opened_at / closed_at: ISO times
    """
    symbol: str
    direction: str
    entry_orders: List[Order] = field(default_factory=list)
    entry_fills: List[Fill] = field(default_factory=list)
    exit_legs: List[TradeLeg] = field(default_factory=list)
    size: float = 0.0
    avg_entry: Optional[float] = None
    avg_exit: Optional[float] = None
    total_fee: float = 0.0
    realized_pnl: float = 0.0
    opened_at: Optional[str] = None
    closed_at: Optional[str] = None

    def to_dict(self) -> Dict[str, Any]:
        return {
            "symbol": self.symbol,
            "direction": self.direction,
            "size": self.size,
            "avg_entry": self.avg_entry,
            "avg_exit": self.avg_exit,
            "total_fee": self.total_fee,
            "realized_pnl": self.realized_pnl,
            "opened_at": self.opened_at,
            "closed_at": self.closed_at,
            "legs": [
                {
                    "label": leg.label,
                    "order_id": leg.order.id,
                    "filled_size": leg.filled_size,
                    "avg_price": leg.avg_price,
                    "fee": leg.fee,
                    "leg_pnl": leg.realized_pnl,
                }
                for leg in self.exit_legs
            ],
        }


# =========================
# Assembler
# =========================

class TradeAssembler:
    """
    Builds Trade objects from raw orders + fills.

    How it works (per symbol):
      1) Sort all orders by created_at ascending, attach their fills (by order_id).
      2) Maintain a running 'position queue' of entry fills (FIFO).
      3) Non-reduce-only orders add to the queue (entries/scale-ins).
      4) Reduce-only orders drain the queue (exits/scale-outs):
         - Each exit order becomes a leg (tp1/tp2/... or sl/manual).
         - FIFO matching computes PnL for that leg.
      5) When the queue returns to zero size → close the Trade (flat).
    """

    def __init__(self, orders: List[dict], fills: List[dict]):
        # Normalize to dataclasses
        self.orders: List[Order] = [Order.from_api(o) for o in orders]
        self.fills: List[Fill] = [Fill.from_api(f) for f in fills]
        # Group fills by order_id
        self.fills_by_order: Dict[str, List[Fill]] = {}
        for f in self.fills:
            self.fills_by_order.setdefault(f.order_id, []).append(f)
        # Sort fills in time within each order
        for oid in self.fills_by_order:
            self.fills_by_order[oid].sort(key=lambda x: _ts(x.created_at))

    def assemble(self) -> List[Trade]:
        trades: List[Trade] = []
        # group by symbol
        by_symbol: Dict[str, List[Order]] = {}
        for o in self.orders:
            by_symbol.setdefault(o.symbol, []).append(o)
        # process each symbol independently
        for symbol, olist in by_symbol.items():
            olist.sort(key=lambda o: _ts(o.created_at))
            trades.extend(self._assemble_symbol(symbol, olist))
        return trades

    # ---------- per-symbol assembly ----------

    def _assemble_symbol(self, symbol: str, orders: List[Order]) -> List[Trade]:
        trades: List[Trade] = []

        # position FIFO queue: list of entry fills with remaining size to match
        # For LONG: entries are BUY fills; exits must be SELL reduce_only
        # For SHORT: entries are SELL fills; exits must be BUY reduce_only
        pos_queue: List[Fill] = []
        current: Optional[Trade] = None
        tp_counter = 0

        def direction_from_order(o: Order) -> Optional[str]:
            if o.reduce_only:
                return None
            if o.side == "buy":
                return "long"
            if o.side == "sell":
                return "short"
            return None

        def infer_leg_label(o: Order, entry_avg: Optional[float]) -> str:
            # SL signal from stop_order_type
            if o.stop_order_type:
                return "sl"
            # Heuristic TP if price is favorable vs entry_avg
            fs = self.fills_by_order.get(o.id, [])
            if not fs or entry_avg is None:
                return "manual"
            avg_exit = _weighted_avg([(f.price, f.size) for f in fs])
            if avg_exit is None:
                return "manual"
            if current and current.direction == "long":
                return "tp" if avg_exit > entry_avg else "manual"
            if current and current.direction == "short":
                return "tp" if avg_exit < entry_avg else "manual"
            return "manual"

        for o in orders:
            ofills = self.fills_by_order.get(o.id, [])

            # 1) ENTRY / SCALE-IN (non-reduce-only)
            if not o.reduce_only:
                dirn = direction_from_order(o)
                if dirn:
                    # start new trade if none active
                    if current is None:
                        current = Trade(symbol=symbol, direction=dirn)
                        tp_counter = 0
                    # If direction flips mid-cycle (edge), close previous (force) and start new
                    elif current.direction != dirn and _sum_size(pos_queue) != 0:
                        # Forced close with no explicit reduce-only exits: mark as manual close at last fill price
                        _finalize_trade_without_exits(current, pos_queue)
                        trades.append(current)
                        current = Trade(symbol=symbol, direction=dirn)
                        tp_counter = 0

                    # push entry fills
                    for f in ofills:
                        if _is_entry_fill(current.direction, f.side):
                            pos_queue.append(f)
                            current.entry_fills.append(f)
                            current.size = max(current.size, abs(_sum_size(pos_queue)))
                            current.total_fee += f.fee
                            if current.opened_at is None or _ts(f.created_at) < _ts(current.opened_at):
                                current.opened_at = f.created_at
                    current.entry_orders.append(o)
                    current.avg_entry = _weighted_avg([(f.price, f.size) for f in current.entry_fills])

            # 2) EXIT / SCALE-OUT (reduce-only)
            else:
                if current is None:
                    # Reduce-only but no active trade in assembler; skip or start a synthetic short/long?
                    # We'll treat as manual isolated leg (open & close immediately) by creating a tiny trade.
                    synthetic = _synthetic_trade_from_reduce_only(symbol, o, ofills)
                    if synthetic:
                        trades.append(synthetic)
                    continue

                # Match fills FIFO and compute PnL contribution
                exit_size = sum(f.size for f in ofills if _is_exit_fill(current.direction, f.side))
                if exit_size <= 0:
                    continue

                # Leg label
                leg_type = infer_leg_label(o, current.avg_entry)
                if leg_type == "tp":
                    tp_counter += 1
                    leg_label = f"tp{tp_counter}"
                else:
                    leg_label = leg_type  # 'sl' or 'manual'

                realized_pnl, fee_sum, matched_fills, exit_avg = _fifo_match_pnl(
                    direction=current.direction,
                    pos_queue=pos_queue,
                    exit_fills=[f for f in ofills if _is_exit_fill(current.direction, f.side)]
                )

                # record leg
                leg = TradeLeg(
                    label=leg_label,
                    order=o,
                    fills=matched_fills,
                    filled_size=sum(f.size for f in matched_fills),
                    avg_price=exit_avg or 0.0,
                    fee=fee_sum,
                    realized_pnl=realized_pnl
                )
                current.exit_legs.append(leg)
                current.total_fee += fee_sum
                current.realized_pnl += realized_pnl

                # If position fully closed → finalize trade
                if _sum_size(pos_queue) == 0:
                    current.avg_exit = _weighted_avg([
                        (leg.avg_price, leg.filled_size) for leg in current.exit_legs if leg.filled_size > 0
                    ])
                    # closed_at = last exit fill time
                    last_exit_ts, last_exit_iso = 0.0, None
                    for L in current.exit_legs:
                        for f in L.fills:
                            ts = _ts(f.created_at)
                            if ts >= last_exit_ts:
                                last_exit_ts, last_exit_iso = ts, f.created_at
                    current.closed_at = last_exit_iso
                    trades.append(current)
                    current = None
                    tp_counter = 0

        # If leftover open position -> leave as open trade (no close legs)
        if current and _sum_size(pos_queue) > 0:
            # keep as open; avg_exit None
            trades.append(current)

        return trades


# =========================
# Helpers
# =========================

def _sum_size(fills: List[Fill]) -> float:
    return sum(f.size for f in fills)

def _weighted_avg(pairs: List[Tuple[float, float]]) -> Optional[float]:
    """
    pairs = [(price, size), ...]
    """
    num = sum(p * s for p, s in pairs)
    den = sum(s for _, s in pairs)
    return (num / den) if den > 0 else None

def _is_entry_fill(direction: str, fill_side: str) -> bool:
    return (direction == "long" and fill_side == "buy") or (direction == "short" and fill_side == "sell")

def _is_exit_fill(direction: str, fill_side: str) -> bool:
    return (direction == "long" and fill_side == "sell") or (direction == "short" and fill_side == "buy")

def _fifo_match_pnl(direction: str, pos_queue: List[Fill], exit_fills: List[Fill]) -> Tuple[float, float, List[Fill], Optional[float]]:
    """
    Match exit fills against pos_queue (entry fills) FIFO to compute realized PnL.

    Returns:
        realized_pnl (USD), fee_sum, matched_exit_fills (possibly split), exit_avg_price
    """
    realized = 0.0
    fee_sum = 0.0
    matched: List[Fill] = []
    weighted_pairs: List[Tuple[float, float]] = []

    # Clone queue sizes we can mutate
    i = 0
    for ex in exit_fills:
        remaining = ex.size
        fee_sum += ex.fee
        while remaining > 1e-12 and i < len(pos_queue):
            en = pos_queue[i]
            take = min(remaining, en.size)
            # Effective prices
            en_px, ex_px = en.price, ex.price
            # direction → sign of pnl
            if direction == "long":
                pnl = (ex_px - en_px) * take
            else:
                pnl = (en_px - ex_px) * take
            realized += pnl
            weighted_pairs.append((ex_px, take))
            matched.append(Fill(
                order_id=ex.order_id, symbol=ex.symbol, side=ex.side,
                price=ex_px, size=take, fee=0.0, created_at=ex.created_at
            ))
            # reduce from entry lot
            en.size -= take
            remaining -= take
            if en.size <= 1e-12:
                i += 1  # consume entry lot completely

    # Remove fully consumed entry lots from queue head
    del pos_queue[:i]
    exit_avg = _weighted_avg(weighted_pairs)
    return realized, fee_sum, matched, exit_avg

def _finalize_trade_without_exits(trade: Trade, pos_queue: List[Fill]) -> None:
    """
    Edge-case helper if direction flips without explicit reduce-only legs.
    We mark everything closed at the last entry fill price (realized PnL = 0).
    """
    if not trade.entry_fills:
        return
    trade.avg_entry = _weighted_avg([(f.price, f.size) for f in trade.entry_fills])
    trade.avg_exit = trade.avg_entry
    trade.closed_at = max((f.created_at for f in trade.entry_fills), default=None)
    trade.realized_pnl += 0.0
    pos_queue.clear()

def _synthetic_trade_from_reduce_only(symbol: str, o: Order, ofills: List[Fill]) -> Optional[Trade]:
    """
    If we see a reduce-only order without an active trade in memory,
    create a synthetic one-order trade (manual). Useful for data gaps.
    """
    if not ofills:
        return None
    # Guess direction by opposite of exit side
    # If exit side is SELL, the open direction was LONG; if BUY, it was SHORT.
    direction = "long" if ofills[0].side == "sell" else "short"
    t = Trade(symbol=symbol, direction=direction)
    # Treat fills as both entry and exit at same price → PnL ~ 0
    for f in ofills:
        t.entry_fills.append(Fill(order_id=f.order_id, symbol=f.symbol, side=("buy" if direction=="long" else "sell"),
                                  price=f.price, size=f.size, fee=0.0, created_at=f.created_at))
        t.exit_legs.append(TradeLeg(label="manual", order=o, fills=[f], filled_size=f.size,
                                    avg_price=f.price, fee=f.fee, realized_pnl=0.0))
        t.total_fee += f.fee
    t.size = _sum_size(t.entry_fills)
    t.avg_entry = _weighted_avg([(f.price, f.size) for f in t.entry_fills])
    t.avg_exit = _weighted_avg([(f.price, f.size) for f in ofills])
    t.opened_at = min((f.created_at for f in t.entry_fills), default=None)
    t.closed_at = max((f.created_at for f in ofills), default=None)
    return t


# =========================
# Example CLI
# =========================

if __name__ == "__main__":
    client = DeltaIndiaClient(api_key=API_KEY, api_secret=API_SECRET)
    # Pull everything; you can add product_ids=[14823] for SOLUSD only
    orders = client.get_all_orders()
    fills = client.get_all_fills()

    assembler = TradeAssembler(orders, fills)
    trades = assembler.assemble()

    print(f"Built {len(trades)} trades")
    for t in trades[:5]:
        print(t.to_dict())


Built 105 trades
{'symbol': 'SOLUSD', 'direction': 'short', 'size': 3.0, 'avg_entry': 183.03599999999997, 'avg_exit': 181.73566666666667, 'total_fee': 0.0, 'realized_pnl': 3.9010000000000105, 'opened_at': '2025-08-18T13:23:41.777908Z', 'closed_at': '2025-08-18T15:50:50.115058Z', 'legs': [{'label': 'sl', 'order_id': '842728902', 'filled_size': 1.0, 'avg_price': 183.007, 'fee': 0.0, 'leg_pnl': 0.028999999999996362}, {'label': 'tp1', 'order_id': '842837963', 'filled_size': 2.0, 'avg_price': 181.1, 'fee': 0.0, 'leg_pnl': 3.872000000000014}]}
{'symbol': 'SOLUSD', 'direction': 'long', 'size': 3.0, 'avg_entry': 182.037, 'avg_exit': 182.40600000000003, 'total_fee': 0.0, 'realized_pnl': 1.1069999999999993, 'opened_at': '2025-08-19T12:13:23.455545Z', 'closed_at': '2025-08-19T12:27:47.067015Z', 'legs': [{'label': 'tp1', 'order_id': '846889757', 'filled_size': 3.0, 'avg_price': 182.40600000000003, 'fee': 0.0, 'leg_pnl': 1.1069999999999993}]}
{'symbol': 'SOLUSD', 'direction': 'short', 'size': 3.0, 